In [1]:
import pandas as pd
import json
from bs4 import BeautifulSoup
import numpy as np
import requests as r

In [2]:
with open('./source/questionnaire.json') as json_file:
    data = json.load(json_file)
    data = data['top_container']

In [3]:
exclude = [43374942,43426483,43374941,43374933,43375020,43374922,43398762]

In [4]:
country_key = 43380772
funds = 43374826
contribution = 43374827

In [5]:
config = {
    "values":{
        "43374939":"Type of Action",
        "43374932":"Role Organisation",
        "43374862":"Responsible Actor",
        "43375025":"Reporting",
        "43374922":"Other Type of Action",
        "43374951":"Reporting and Evaluations",
        "43374934":"Outcome Evaluated",
        "43374949":"Geography",
        "43374915":"Source to Sea",
        "43374916":"Lifecycle of Plastics",
        "43374904":"Target Action",
        "43374931":"Impact",
        "43374917":"Pollutant Targeted",
        "43374905":"Sector",
        "43374920":"Funding",
        "43374943":"Duration",
        "43374933":"First Time / Returning",
    },
    "country":{
        "id": "43380772"
    }
}

In [6]:
all_childs = []

def getAllChilds(x, pid=None, p=None):
    qid = None
    text = None
    dependencies = None
    if 'q_no' in x:
        qid = str(x['q_no'])
    if p:
        if 'q_no' in p:
            qid = str(p['q_no'])
        if 'q_no' in x:
            qid = str(x['q_no'])
    if 'text' in x:
        text = x['text']
        if text == " ":
            text = p['text']
    if 'title' in x:
        text = x['title']
    if pid:
        pid = str(pid)
    if 'dependencies' in x:
        dependencies = json.loads(x['dependencies'])
        dependencies = dependencies[0]
        rule = int(dependencies['rule'])
        parent = [o for o in p['options'] if o['value'] == rule]
        pid = str(parent[0]['id'])
    if text:
        text = BeautifulSoup(text)
        text = text.get_text()
        all_childs.append({
            "id": x['id'],
            "name": text.replace('\xa0',' ').replace('\n','').replace('  ',' '),
            "pid": pid,
            "qid": qid,
            "type": x["class"],
        })
    if 'options' in x:
        for z in x['options']:
            getAllChilds(z, x['id'], x)
    if 'children' in x:
        for z in x['children']:
            getAllChilds(z, x['id'], x)

In [7]:
for d in data['children']:
    getAllChilds(d)

In [8]:
df = pd.DataFrame(all_childs)

In [9]:
df = df[~df['type'].isin(['Page','Section','Note'])]

In [10]:
df['qpar'] = df['qid'].apply(lambda x: x.split('.')[0]).astype('int')

In [11]:
df.to_csv('test.csv')

In [12]:
resp = pd.read_csv('./source/data-production_2020-08-03.csv')

In [13]:
q_rename = {}
for qs in list(resp):
    qid = qs.split(' ')[0]
    if qid[-1] == ".":
        qid = qid[:-1]
        qname = '. '.join(qs.split('. ')[1:])
        ids = df[(df['qid']==qid)]
        ids = ids[~(ids['type']=='SelectionOption')]
        try:
            if qname in list(ids['name'])[0]:
                qid = list(ids['id'])[0]
        except:
            pass
    q_rename.update({qs:qid})

In [14]:
records = resp.rename(columns=q_rename)

In [18]:
records = records.drop(exclude, axis=1)

In [19]:
records

,Unique,Introduction,43374842,43375025,43437497,43374845,43375020,43374800,43374829,43374939,...,43412006,43374810,43374839,43374835,43374837,43374822,43374823,43374811,StartDate,CompletionDate
0,545728-545719-53376637,NaN,The SeaCleaners,On behalf of an organisation,NaN,The SeaCleaners,4.0,AWARENESS RAISING,#NAME?,WORKING WITH PEOPLE: Encouraging or enabling o...,...,NaN,https://www.theseacleaners.org/en/educational-...,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-20 17:36:36 UTC,2019-12-30 15:42:40 UTC
1,545728-545719-53649934,NaN,The Institute of Environmental Science and Res...,As an individual,NaN,NaN,4.0,New Zealand Plastic bag ban,Government ban of the use of single-use plasti...,"LEGISLATION, STANDARDS, RULES: e.g. agreeing n...",...,NaN,https://www.mfe.govt.nz/waste/single-use-plast...,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-07 22:32:59 UTC,2020-01-07 23:05:20 UTC
2,545728-545719-53668499,NaN,"Secretariat of the Basel, Rotterdam and Stockh...",On behalf of an organisation,NaN,"Secretariat of the Basel, Rotterdam and Stockh...",3.0,Updating the technical guidelines for the iden...,"At its sixth meeting held in December 2002, th...","LEGISLATION, STANDARDS, RULES: e.g. agreeing n...",...,NaN,http://www.basel.int/Implementation/Plasticwas...,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-08 13:54:19 UTC,2020-01-10 15:05:07 UTC
3,545728-545719-53741002,NaN,Dirección del Parque Nacional Galápagos,On behalf of an organisation,NaN,Dirección del Parque Nacional Galápagos,1.0,LIMPIEZA COSTERA EN GALÁPAGOS,Se realizan viajes en diferentes embarcaciones...,WORKING WITH PEOPLE: Encouraging or enabling o...,...,NaN,http://www.galapagos.gob.ec/46-toneladas-de-ba...,http://www.ambiente.gob.ec/limpieza-costera-un...,http://www.galapagos.gob.ec/galapagos-se-sumo-...,https://conservation.org.ec/boletines/iniciati...,NaN,NaN,NaN,2020-01-10 14:06:52 UTC,2020-01-10 16:21:59 UTC
4,545728-545719-53853866,NaN,"Institute of Oceanography and Fisheries, Split...",As an individual,NaN,NaN,1.0,Fishing for litter,Fisheries is a sector that can make a signific...,WORKING WITH PEOPLE: Encouraging or enabling o...,...,NaN,http://www.ml-repair.eu/en,http://www.defishgear.net/desert2/itemlist/tag...,https://www.jutarnji.hr/vijesti/hrvatska/alarm...,https://net.hr/danas/hrvatska/ponosni-ste-na-p...,http://www.ml-repair.eu/attachment/preview/5dc...,https://komunalne-djelatnosti.hr/wp-content/up...,"Before the DeFishGear project started in 2013,...",2020-01-14 12:48:24 UTC,2020-01-14 13:33:37 UTC
5,545728-545719-53855495,NaN,UNEP,On behalf of an organisation,NaN,UNEP Law Division,1.0,legislative guidance to countries on marine li...,legislative assistance to countries,"LEGISLATION, STANDARDS, RULES: e.g. agreeing n...",...,NaN,https://www.unenvironment.org/resources/report...,NaN,NaN,NaN,NaN,NaN,#NAME?,2020-01-14 13:15:32 UTC,2020-01-14 14:43:51 UTC
6,545728-545719-53915703,NaN,National Geographic Society,On behalf of an organisation,NaN,National Geographic Society,2.0,'Sea to Source: Ganges' Expedition,The “Sea to Source: Ganges” river expeditions ...,MONITORING and ANALYSIS: Collecting evidence a...,...,To be determined - likely continuous over 3+ y...,https://www.nationalgeographic.org/projects/pl...,https://www.nationalgeographic.org/projects/pl...,NaN,NaN,NaN,NaN,NaN,2020-01-15 19:44:36 UTC,2020-01-15 20:23:41 UTC
7,545728-545719-53934963,NaN,Clean Up Kenya,On behalf of an organisation,NaN,Clean Up Kenya,1.0,Ban the bottle campaign,The Ban the Bottle Campaign is designed to pre...,WORKING WITH PEOPLE: Encouraging or enabling o...,...,NaN,https://www.facebook.com/notes/clean-up-kenya/...,https://www.facebook.com/1553146668276531/post...,https://www.facebook.com/1553146668276531/post...,https://www.facebook.com/1553146668276531/post...,NaN,NaN,NaN,2020-01-16 11:15:57 UTC,2020-01-16 12:00:32 UTC
8,545728-545719-53944718,NaN,National Environment Commission,As an individual,NaN,NaN,1.0,Survey in marine litter and microplactics,"Regulatory, implementation, enforcement","LEGISLATION, STANDARD

In [20]:
records[country_key] = records[country_key].apply(lambda x: "Côte d'Ivoire" if "Cote d'Ivoire" in x else x)

In [21]:
pd.DataFrame(records[43374842].str.strip().unique(), columns=["organisation_names"]).to_csv('organisationlist.csv')

In [22]:
non_var = ['StartDate','CompletionDate','Unique', country_key]
results = []
for a in records.to_dict('records'):
    for b in a:
        if b not in non_var:
            opt = df[(df['pid'] == str(b)) & (df['type'] == 'SelectionOption')]
            values = []
            for c in opt.to_dict('records'):
                if c['name'] in str(a[b]):
                    values.append(c['id'])
            if len(values) == 0:
                values = np.nan
            if values == values:
                results.append({'uuid':a['Unique'],'qid':b,'values':values,'countries':a[country_key]})

In [23]:
res = pd.DataFrame(results)

In [24]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, query
import numpy as np
from app.models import Countries, Groups, Values, CountryGroups, Datapoints, DatapointCountries, DatapointValues
from app.connection import engine_url, write_data

In [25]:
engine_url = engine_url()
engine = create_engine(engine_url)
session = sessionmaker(bind=engine)()

In [26]:
values = df.rename(columns={'id':'code','pid':'parent_code'})
values = values[['name','code','parent_code','type']]

In [27]:
question_list = list(res['qid'].unique())

In [28]:
for r in res['values'].to_list():
    for z in r:
        if z not in question_list:
            question_list.append(z)

In [29]:
values = values[values['code'].isin(question_list)]

In [30]:
def renameColumn(x):
    name = x['name']
    try:
        name = config['values'][str(x['code'])]
    except:
        pass
    return name

In [31]:
values['name'] = values.apply(lambda x: renameColumn(x), axis=1)

In [32]:
def get_parent(x):
    parent_id = x['parent_code']
    if parent_id:
        parent = values[values['code'] == int(parent_id)]
        if parent.empty:
            return None
    return x['parent_code']

In [33]:
def get_grand_parent(x):
    parent_id = x['parent_code']
    if parent_id:
        parent = values[values['code'] == int(parent_id)]
        if not parent.empty:
            parent_id = parent.to_dict('records')[0]['parent_code']
            if parent_id:
                parent = values[values['code'] == int(parent_id)]
                if not parent.empty:
                    try:
                        return parent['code'][0]
                    except:
                        return parent_id
    return parent_id

In [34]:
values['parent_code'] = values.apply(get_parent, axis=1)
values['grand_parent'] = values.apply(get_grand_parent, axis=1)
values = values.reset_index()

In [35]:
excludes = values[(values['type'] == 'MultipleAnswer') & (values['parent_code'])]
excludes = excludes['index'].to_list()

In [36]:
values = values[~values['index'].isin(excludes)]
values['parent_code'] = values.apply(lambda x: x['grand_parent'] if x['grand_parent'] else x['parent_code'], axis=1)

In [37]:
for value in values.to_dict('records'):
    run = True
    if value['type'] == 'SelectionList' and value['parent_code'] != None:
        run = False
    if run:
        value.update({'parent_id':None,'description':None})
        parent = session.query(Values).filter(Values.code == value['parent_code']).first()
        if parent:
            value.update({'parent_id':parent.id})
        input_data = Values(value)
        write_data(session, input_data, value, False)

In [38]:
for uuid in set(list(records['Unique'])):
    rec = records.fillna(0)
    colval = rec[rec['Unique'] == uuid][[funds, contribution]]
    fund = int(colval.to_dict('records')[0][funds])
    contrib = int(colval.to_dict('records')[0][contribution])
    datapoint = {'uuid':uuid,'funds':fund,'contribution':contrib}
    input_data = Datapoints(datapoint)
    write_data(session, input_data, datapoint, False)

In [39]:
for dc in records[['Unique', country_key]].to_dict('records'):
    datapoint = session.query(Datapoints).filter(Datapoints.uuid == dc['Unique']).first()
    if dc[country_key] == dc[country_key]:
        if "All" not in dc[country_key]:
            for c in dc[country_key].split(','):
                country = session.query(Countries).filter(Countries.name == c).first()
                if country:
                    datapoint_country = {
                        'datapoint_id':datapoint.id,
                        'country_id': country.id
                    }
                    input_data = DatapointCountries(datapoint_country)
                    write_data(session, input_data, datapoint_country, False)
        else:
            countries = session.query(Countries).all()
            for country in countries:
                datapoint_country = {
                    'datapoint_id':datapoint.id,
                    'country_id': country.id
                }
                input_data = DatapointCountries(datapoint_country)
                write_data(session, input_data, datapoint_country, False)

In [40]:
for values in res.to_dict('records'):
    datapoint = session.query(Datapoints).filter(Datapoints.uuid == values['uuid']).first()
    for v in values['values']:
        value = session.query(Values).filter(Values.code == v).first()
        if value:
            datapoint_value = {
                'datapoint_id':datapoint.id,
                'value_id': value.id
            }
            input_data = DatapointValues(datapoint_value)
            write_data(session, input_data, datapoint_value, False)